In [11]:
# cow.py
class CowHangman:
    def __init__(self):
        self.lives = 6

        # Define the cow's parts in an accumulative manner
        self.cow_parts = [
            '''
             ^__^
            ''',
            '''
             ^__^
             (oo)
            ''',
            '''
             ^__^
             (oo)\_______
            ''',
            '''
             ^__^
             (oo)\_______
             (__)\       
            ''',
            '''
             ^__^
             (oo)\_______
             (__)\       )\/\\
            ''',
            '''
             ^__^
             (oo)\_______
             (__)\       )\/\\
                 ||----w |
                 ||     ||
            '''
        ]
        
    def display_cow(self):
        parts_to_display = min(5, 6 - self.lives)  # This ensures that the maximum value of parts_to_display is 5
        print(self.cow_parts[parts_to_display])


    def lose_life(self):
        if self.lives > 0:
            self.lives -= 1
            self.display_cow()
        else:
            print("No more lives!")
    
    def is_game_over(self):
        return self.lives <= 0


In [13]:
import sys
import pandas as pd

df = pd.read_csv("Database.csv")
word_database = df['Word'].tolist()
#word_database = ["WELLMAN HALL", "ARC", "ART", "APT", "APP", "ATV", "YOUNG HALL", "KEMPER HALL", "AGGIES", "GO AGS", "PAVILLION", "eureka", "mathematics", "physics"]
game_over = False

def compute_letter_weights(word_database):
    total_letters = 0
    letter_counts = {char: 0 for char in 'abcdefghijklmnopqrstuvwxyz'}
    
    for word in word_database:
        word = word.lower()  # Ensure the word is in lowercase for consistency
        for letter in word:
            if letter in letter_counts:
                letter_counts[letter] += 1
                total_letters += 1

    # Normalize to get frequencies
    letter_frequencies = {letter: (count / total_letters) * 100 for letter, count in letter_counts.items()}
    
    return letter_frequencies

# Compute the weights at the start of the code
letter_weights = compute_letter_weights(word_database)

class HangmanGame:
    def __init__(self, word_length):
        self.word_length = word_length
        self.state = "_" * word_length
        self.guesses = []

    def get_state(self):
        return self.state

    def guess(self, letter):
        self.guesses.append(letter)
        if letter in self.state:
            return True
        return False

    def update_state(self, positions, letter):
        state_list = list(self.state)
        for pos in positions:
            state_list[pos] = letter
        self.state = ''.join(state_list)


class EntropyBasedPlayer:
    def __init__(self, word_database):
        self.word_database = word_database
        self.already_guessed = []

    def filter_words(self, word_length):
        return [word for word in self.word_database if len(word) == word_length]

    def next_guess(self, current_state):
        potential_matches = [word for word in self.filter_words(len(current_state)) if self.matches_state(word, current_state)]
    
        # If only one potential word match is left
        if len(potential_matches) == 1:
            return potential_matches[0]
    
        next_underscore_position = current_state.index("_")
    
        frequency_distribution = {}
        for word in potential_matches:
            letter = word[next_underscore_position]
            if letter not in self.already_guessed:
                frequency_distribution[letter] = frequency_distribution.get(letter, 0) + 1

        if not frequency_distribution:
            return None

        # Prioritize based on count first, and then by letter weight in case of a tie.
        guess = max(frequency_distribution, key=lambda k: (frequency_distribution[k], letter_weights.get(k, 0)))
        self.already_guessed.append(guess)
        return guess
    

    def matches_state(self, word, state):
        for w, s in zip(word, state):
            if s != '_' and w != s:
                return False
        return True

    def reset_guessed(self):
        self.already_guessed = []
try:
    if __name__ == "__main__" and not game_over:
        cow_game = CowHangman()
        word_length = int(input("Enter the length of the word: "))
        hangman = HangmanGame(word_length)
        player = EntropyBasedPlayer(word_database)

        while "_" in hangman.get_state():
            print("Current state:", hangman.get_state())
            guess = player.next_guess(hangman.get_state())

            # If the guess is a complete word
            if len(guess) == word_length:
                is_word = input(f"Is the word {guess}? (y/n): ").lower()
                if is_word == 'y':
                    print("AI won!")
                    sys.exit()
                else:
                    print("AI lost!")
                    sys.exit()

            # If the AI cannot guess anymore
            if guess is None:
                print("AI is out of guesses!")
                if cow_game.is_game_over():
                    sys.exit()

            print("AI guesses:", guess)
            is_correct = input("Is the guess correct? (y/n): ").lower()
            if is_correct == 'y':
                positions = []
                word_list = list(hangman.get_state())
                for idx, char in enumerate(word_list):
                    if char == "_":
                        word_list[idx] = guess
                        positions.append(idx)
                        break
                hangman.update_state(positions, guess)
                player.reset_guessed()
            else:
                print("Guess is incorrect.")
                cow_game.lose_life()
                if cow_game.is_game_over():
                    print("AI lost!")
                    sys.exit()

        if "_" not in hangman.get_state():
            print("AI won!")
            sys.exit()
        else:
            print("AI lost!")
            sys.exit()
except SystemExit:
    pass

Enter the length of the word: 5
Current state: _____
AI guesses: l
Is the guess correct? (y/n): y
Current state: l____
AI guesses: a
Is the guess correct? (y/n): y
Current state: la___
Is the word laben? (y/n): y
AI won!
